In [ ]:
import torch
import torch.nn as nn

In [ ]:
import gymnasium as gym

env = gym.make("ALE/Kaboom-v5", render_mode='rgb_array')
from nn_model6 import DQNEfficientNet
model = DQNEfficientNet(env.action_space.n)

wei = torch.load("trains/model_480.pth", weights_only=True)
model.load_state_dict(wei)
for param in model.parameters():
    print(f'number of parameters:{param.numel(): 12d} {param.shape}')

In [ ]:
import matplotlib.pyplot as plt
fig, axs =  plt.subplots(4,8,figsize=(10, 5))
for c in range(32):
    with torch.no_grad():
        max = model.features[0].weight[c].max().item()
        min = model.features[0].weight[c].min().item()
        img_norm = (model.features[0].weight[c].permute(1,2,0).numpy()-min)/(max - min)
        # print(max, min)
        axs[c//8][c%8].imshow(img_norm)

In [ ]:
import cv2
state,_ = env.reset()
def obs2stateTensor(obs, show=False):
    # obs = obs[:, 420:1500] # choose area for model input
    # state = cv2.resize(obs, (224, 224))
    state = obs
    if(show): 
        cv2.imshow('model input', cv2.cvtColor(state, cv2.COLOR_RGB2BGR))
        cv2.waitKey(1)
    state = torch.FloatTensor(state)
    state = state.permute(2, 0, 1).contiguous()  # (H, W, C) -> C, H, W
    state = (state - state.mean())/state.std() # normalize
    return state

In [ ]:
img_axs = plt.subplot()
fig, axs =  plt.subplots(8,8,figsize=(10, 10))
with torch.no_grad():
    state, reward, done,trancated, info = env.step(env.action_space.sample())
    img_axs.imshow(state)
    state = obs2stateTensor(state, show=False)
    afterCNN = model.features[0:8](state)
    print(afterCNN.shape)
    for c in range(16):
        axs[c//8][c%8].imshow(afterCNN[c], cmap='gray')

In [ ]:
state,_ = env.reset()

In [ ]:
# img_axs = plt.subplot()
fig, axs =  plt.subplots(1,2,figsize=(16, 5), gridspec_kw={'width_ratios': [1, 4]})

with torch.no_grad():
    state, reward, done,trancated, info = env.step(env.action_space.sample())
    axs[0].imshow(state)
    state = obs2stateTensor(state, show=False)
    afterCNN = model.features[0:6](state)
    #normalize each img
    max = afterCNN.amax(dim= (-1,-2), keepdim=True)
    min = afterCNN.amin(dim= (-1,-2), keepdim=True)
    afterCNN = (afterCNN - min)/(max - min + 1e-8)    
    afterCNN = torch.nn.functional.pad(afterCNN, (0, 1, 0, 1), mode='constant',value=1) # pad to add border
    print(afterCNN.shape)
    img_w = afterCNN.shape[-1]
    img_h = afterCNN.shape[-2]
    img_c = afterCNN.shape[-3]
    afterCNN = afterCNN.reshape(-1,8,img_h,img_w).permute(0,2,1,3).reshape(-1,8*img_w)
    print(afterCNN.shape)
    axs[1].imshow(afterCNN, cmap='gray')
    # print(cv2.resize((afterCNN.unsqueeze(-1)*torch.ones(3)).numpy(), (420, afterCNN.shape[-1]*320/afterCNN.shape[-2])).shape)
    
    # for c in range(16):
    #     axs[c//8][c%8].imshow(afterCNN[c], cmap='gray')

In [ ]:
model

In [ ]:
# Play the game use the model and display interal CNN features
import time
import numpy as np
cnn_layer = 8
num_img_horiz = 2
obs,_ = env.reset()
for i in range(10000000):
    with torch.no_grad():
        state = obs2stateTensor(obs, show=False)
        q_values = model(state.unsqueeze(0))
        action = q_values.argmax().item() 
        obs, reward, done,trancated, info = env.step(action)
        if(done):
            obs,_ = env.reset()
            print('Started new game')
            continue
        afterCNN = model.features[0:cnn_layer](state)
        #normalize each img
        # max = afterCNN.amax(dim= (-1,-2), keepdim=True)
        # min = afterCNN.amin(dim= (-1,-2), keepdim=True)
        max = afterCNN.max()
        min = afterCNN.min()
        afterCNN = (afterCNN - min)/(max - min + 1e-8)    
        afterCNN = torch.nn.functional.pad(afterCNN, (0, 1, 0, 1), mode='constant',value=1) # pad to add border
        # print(afterCNN.shape)
        img_w = afterCNN.shape[-1]
        img_h = afterCNN.shape[-2]
        img_c = afterCNN.shape[-3]
        afterCNN = afterCNN.reshape(-1,num_img_horiz,img_h,img_w).permute(0,2,1,3).reshape(-1,num_img_horiz*img_w)
        # print(afterCNN.shape)
        # print((afterCNN).max())
        state = cv2.cvtColor(obs.astype(np.float32)/200, cv2.COLOR_RGB2BGR)
        cv_img = cv2.hconcat([cv2.resize(state,(640,840), interpolation=cv2.INTER_AREA), 
                            cv2.resize((afterCNN.unsqueeze(-1)*torch.ones(3)).numpy(), (int(afterCNN.shape[-1]*840/afterCNN.shape[-2]), 840)
                                       ,interpolation=cv2.INTER_AREA)])
        cv2.imshow('Game Analysis', cv_img)
        cv2.waitKey(1)
        time.sleep(0.05)

In [ ]:
import torch

theta = torch.tensor(0., requires_grad=True)
X = torch.tensor([0,1,2], requires_grad=False)
with torch.no_grad():
    probs_dist = torch.distributions.Categorical((X+theta)/(3*theta+3))
    X_samples = probs_dist.sample([10000])
    # probs_dist.log_prob(torch.tensor([0,1,2]))

X_samples = probs_dist.sample([1000000]) # use monte carlo to estimate expectation
def p(x, theta):
    return (x+theta)/(3*theta+3) #p(x|theta)
# print(X_samples)
dd = torch.log(p(X_samples, theta))*X_samples #ln(p(x|theta))*x
(dd.mean()).backward() # backward to get the gradient of ln(p(x|theta))*x, mean is to estimate expectation
print(theta.grad) # the gradient, should be -2/3


In [279]:
import gymnasium as gym
from gymnasium.wrappers import frame_stack,AtariPreprocessing

env = gym.make("ALE/Pong-v5")  # Create an Atari Pong environment

env = AtariPreprocessing(env, grayscale_obs = True,scale_obs=True)
env = frame_stack.FrameStack(env, 4)
# env = wrap_deepmind(env, frame_stack=True, scale=True)  # Apply Atari preprocessing


# Now, the environment provides preprocessed observations
# observation = env.reset()


In [ ]:
observation = env.step(env.action_space.sample())

In [ ]:
import matplotlib.pyplot as plt
observation = env.step(env.action_space.sample())
plt.imshow(observation[0], cmap='gray')